This is a notebook designed to create an FNO which matches evolved turbulence simulations to their forcing function

In [1]:
# imports
import torch
import matplotlib.pyplot as plt
import sys
import jax.numpy as jnp
import numpy as np
from neuralop.models import FNO
from torch.utils.data import DataLoader, TensorDataset
from neuralop import Trainer
from neuralop.training import AdamW
from neuralop.utils import count_model_params
from sklearn.model_selection import train_test_split
from neuralop import LpLoss, H1Loss
from named_dataset import NamedTensorDataset

device = 'cpu'

In [ ]:
# load data
data_path = "/work/09694/carsonmcvay/ls6/turbulence_encryption/forcing_functions/multi_forcing_simulations_combined_viscneg2.npz"
data = jnp.load(data_path)

In [ ]:
# designate inputs and outputs
# inputs are simulations, outputs are forcings
inputs = data["inputs"]
inputs = jnp.fft.irfft(inputs, axis=-1)  # Perform inverse FFT along the last axis # adding this to try to debug
outputs = data["outputs"]

print("Transformed Inputs Shape:", inputs.shape)

Split training and test data

In [ ]:
# split
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

In [ ]:
# Convert data from JAX arrays to NumPy
# doing this before passing to PyTorch
X_train_np = np.array(jnp.asarray(X_train).block_until_ready())  # Ensures conversion to NumPy
y_train_np = np.array(jnp.asarray(y_train).block_until_ready())
X_test_np = np.array(jnp.asarray(X_test).block_until_ready())
y_test_np = np.array(jnp.asarray(y_test).block_until_ready())

In [ ]:
# data loaders
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.float32)

# Create data loaders
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_dataset = NamedTensorDataset(X_train_tensor, y_train_tensor)
test_dataset = NamedTensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# data_processor = data_processor.to(device)

Creat FNO model

In [ ]:
model = FNO(n_modes=(16,16),
           in_channels=1,
           out_channels=1, 
           hidden_channels=32,
           projection_channel_ratio=2)

model = model.to(device)

n_params = count_model_params(model)
print(f"our model has {n_params} parameters")
sys.stdout.flush()

Training setup

In [ ]:
# create optimizer
optimizer = AdamW(model.parameters(),
                 lr=1e-4,
                 weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

# create losses
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)

train_loss = h1loss
eval_losses={'h1': h1loss, 'l2': l2loss}

Training the model

In [ ]:
# create optimizer
optimizer = AdamW(model.parameters(),
                 lr=1e-4,
                 weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

# create losses
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)

train_loss = h1loss
eval_losses={'h1': h1loss, 'l2': l2loss}

In [ ]:
# create the trainer
trainer = Trainer(model=model, n_epochs=20, 
                 wandb_log=False,
                 device=device,
                 mixed_precision=False,
                 data_processor=None,
                 eval_interval=3,
                 log_output=False,
                 use_distributed=False,
                 verbose=True)

In [ ]:
# train model on data
trainer.train(train_loader=train_loader, 
             test_loaders={"default": test_loader},
             optimizer=optimizer,
             scheduler=scheduler,
             regularizer=False,
             training_loss=train_loss,
             eval_losses=eval_losses)